## Early stage Classification of Diabetes

In [141]:
#importing the packages
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from lime.lime_tabular import LimeTabularExplainer
import matplotlib
matplotlib.use('Agg')  # Use non-interactive Agg backend
import matplotlib.pyplot as plt
import plotly.express as px
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [67]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [148]:
try:
    from google.colab import files
    print("Running in Colab. Please upload diabetes.csv")
    uploaded = files.upload()  # Upload diabetes.csv
    data = pd.read_csv('diabetes.csv')
except ImportError:
    # For local Jupyter: Load from path
    data = pd.read_csv(r"C:\Users\kk382\OneDrive\Desktop\dl_project\Diabetes_prediction\diabetes.csv")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define column names
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 
           'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Verify column names
if list(data.columns) != columns:
    raise ValueError(f"Expected columns {columns}, but got {list(data.columns)}")

# Visualize class distribution
gendis = px.histogram(data, x='Outcome', color='Outcome', title="Positive/Negative count Vs Outcome")
gendis.write_html("diabetes_class_distribution.html")  # Save Plotly figure to file

# Handle missing values
missing_columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
data[missing_columns] = data[missing_columns].replace(0, np.nan)
data.fillna(data.mean(), inplace=True)

Using device: cpu


In [149]:
# Separate features and target
X = data.drop('Outcome', axis=1).values
y = data['Outcome'].values


In [150]:

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [151]:
# Implement Proximity-Weighted Synthetic Oversampling (ProWSyn)
def prowsyn(X, y, beta=2.0, K=5, L=3, theta=0.5):
    from sklearn.neighbors import NearestNeighbors
    minority_class = 1
    majority_class = 0
    min_indices = np.where(y == minority_class)[0]
    maj_indices = np.where(y == majority_class)[0]
    X_min = X[min_indices]
    X_maj = X[maj_indices]
    
    ml = len(X_maj)
    ms = len(X_min)
    G = int((ml - ms) * beta)
    print(f"ProWSyn: G={G}, ml={ml}, ms={ms}, beta={beta}")
    
    if G <= 0:
        print("No synthetic samples needed (G <= 0). Returning original data.")
        return X, y
    
    P = X_min.copy()
    partitions = []
    proximity_levels = np.zeros(len(X_min))
    
    for i in range(L-1):
        nn = NearestNeighbors(n_neighbors=min(K, len(P))).fit(P)
        distances, indices = nn.kneighbors(X_maj)
        Pi_indices = np.unique(indices.flatten())
        Pi = P[Pi_indices]
        partitions.append(Pi)
        proximity_levels[Pi_indices] = i + 1
        P = np.delete(P, Pi_indices, axis=0)
    
    if len(P) > 0:
        partitions.append(P)
        remaining_indices = np.where(proximity_levels == 0)[0]
        proximity_levels[remaining_indices] = L
    
    weights = np.exp(-theta * (proximity_levels - 1))
    weights = weights / weights.sum()
    
    synthetic_samples = []
    for idx, x in enumerate(X_min):
        gx = max(int(weights[idx] * G), 1)  # Ensure at least one sample per instance
        for _ in range(gx):
            y_idx = np.random.choice(len(X_min))
            y_sample = X_min[y_idx]
            alpha = np.random.uniform(0, 1)
            s = x + alpha * (y_sample - x)
            synthetic_samples.append(s)
    
    if not synthetic_samples:
        print("No synthetic samples generated. Returning original data.")
        return X, y
    
    synthetic_samples = np.array(synthetic_samples)
    if synthetic_samples.ndim == 1:
        synthetic_samples = synthetic_samples.reshape(-1, X.shape[1])
    
    if synthetic_samples.shape[1] != X.shape[1]:
        print(f"Error: Mismatched dimensions - X shape: {X.shape}, synthetic_samples shape: {synthetic_samples.shape}")
        return X, y
    
    print(f"Generated {len(synthetic_samples)} synthetic samples")
    X_synthetic = np.vstack([X, synthetic_samples])
    y_synthetic = np.hstack([y, [minority_class] * len(synthetic_samples)])
    
    return X_synthetic, y_synthetic


In [152]:
# Apply ProWSyn
print("Original class distribution:", Counter(y))
X_balanced, y_balanced = prowsyn(X, y, beta=2.0, K=5, L=3, theta=0.5)
y_balanced = y_balanced.astype(int)
print("Balanced class distribution:", Counter(y_balanced))

# Verify sufficient samples for stratification
min_samples_per_class = min(Counter(y_balanced).values())
if min_samples_per_class < 10:
    raise ValueError(f"Insufficient samples ({min_samples_per_class}) for 10-fold stratification.")

Original class distribution: Counter({np.int64(0): 500, np.int64(1): 268})
ProWSyn: G=464, ml=500, ms=268, beta=2.0
Generated 268 synthetic samples
Balanced class distribution: Counter({np.int64(1): 536, np.int64(0): 500})


In [153]:
# Define Highway Layer
class HighwayLayer(nn.Module):
    """Highway Layer for adaptive information flow"""
    def __init__(self, units):
        super(HighwayLayer, self).__init__()
        self.transform_gate = nn.Linear(units, units)
        self.transform = nn.Linear(units, units)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        T = self.sigmoid(self.transform_gate(x))
        H = self.relu(self.transform(x))
        C = 1.0 - T
        return H * T + x * C

# Define Highway Model
class HighwayModel(nn.Module):
    def __init__(self, input_dim):
        super(HighwayModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.highway1 = HighwayLayer(64)
        self.highway2 = HighwayLayer(32)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.highway1(x)
        x = self.relu(self.fc2(x))
        x = self.highway2(x)
        x = self.sigmoid(self.fc3(x))
        return x

# Define LeNet Model
class LeNetModel(nn.Module):
    def __init__(self, input_dim):
        super(LeNetModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 6, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(6, 16, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool1d(kernel_size=1)
        self.conv3 = nn.Conv1d(16, 1, kernel_size=1)
        self.fc1 = nn.Linear(input_dim, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.tanh(self.conv1(x))
        x = self.pool1(x)
        x = self.tanh(self.conv2(x))
        x = self.pool2(x)
        x = self.tanh(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Define TCN Model
class TCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super(TCNLayer, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, 
                              padding=(kernel_size-1)*dilation, dilation=dilation)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x

class TCNModel(nn.Module):
    def __init__(self, input_dim, num_layers=2, filters=32, kernel_size=3, dilation_base=2):
        super(TCNModel, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            in_channels = 1 if i == 0 else filters
            self.layers.append(TCNLayer(in_channels, filters, kernel_size, dilation_base**i))
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(filters * input_dim, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.unsqueeze(1)
        for layer in self.layers:
            x = layer(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Define Hi-Le Model
class HiLeModel(nn.Module):
    def __init__(self, input_dim):
        super(HiLeModel, self).__init__()
        self.highway = HighwayModel(input_dim)
        self.lenet = LeNetModel(input_dim)
        self.fc = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        hwy_out = self.highway(x)
        lenet_out = self.lenet(x)
        combined = torch.cat([hwy_out, lenet_out], dim=1)
        out = self.sigmoid(self.fc(combined))
        return out


In [155]:

# Training and Evaluation Functions
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data).squeeze()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        scheduler.step(avg_loss)
        if (epoch + 1) % 5 == 0:  # Log every 5 epochs
            print(f"Epoch {epoch + 1}/{num_epochs}, Avg Loss: {avg_loss:.4f}")

def evaluate_model(model, X_test, y_test):
    model.eval()
    X_test_tensor = torch.FloatTensor(X_test).to(device)
    with torch.no_grad():
        outputs = model(X_test_tensor).squeeze()
        preds = (outputs > 0.5).float().cpu().numpy()
    return accuracy_score(y_test, preds), f1_score(y_test, preds, zero_division=0), \
           precision_score(y_test, preds, zero_division=0), recall_score(y_test, preds, zero_division=0)


In [163]:
# 10-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
metrics_hile = {'accuracy': [], 'f1': [], 'precision': [], 'recall': []}
metrics_hitcle = {'accuracy': [], 'f1': [], 'precision': [], 'recall': []}

for fold, (train_idx, test_idx) in enumerate(skf.split(X_balanced, y_balanced)):
    try:
        print(f"\nFold {fold + 1}/10")
        X_train, X_test = X_balanced[train_idx], X_balanced[test_idx]
        y_train, y_test = y_balanced[train_idx], y_balanced[test_idx]
        
        y_train = y_train.astype(int)
        y_test = y_test.astype(int)
        
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
        )
        
        if len(X_train) == 0 or len(X_val) == 0 or len(X_test) == 0:
            print(f"Warning: Empty split in fold {fold + 1}. Skipping fold.")
            continue
        
        train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
        
        # Train Hi-Le model
        hile_model = HiLeModel(input_dim=X.shape[1]).to(device)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(hile_model.parameters(), lr=0.002)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
        print("Training HiLeModel")
        train_model(hile_model, train_loader, criterion, optimizer, scheduler, num_epochs=20)
        
        # Evaluate Hi-Le
        acc, f1, prec, rec = evaluate_model(hile_model, X_test, y_test)
        metrics_hile['accuracy'].append(acc)
        metrics_hile['f1'].append(f1)
        metrics_hile['precision'].append(prec)
        metrics_hile['recall'].append(rec)
        print(f"HiLeModel Fold {fold + 1} - Accuracy: {acc:.4f}, F1: {f1:.4f}")
        
        # Train base models for HiTCLe
        base_models = [
            (HighwayModel(input_dim=X.shape[1]).to(device), "HighwayModel"),
            (LeNetModel(input_dim=X.shape[1]).to(device), "LeNetModel"),
            (TCNModel(input_dim=X.shape[1], filters=32).to(device), "TCNModel")
        ]
        
        successful_models = []
        base_preds_val = []
        base_preds_test = []
        
        for model, model_name in base_models:
            try:
                print(f"Training {model_name} for fold {fold + 1}")
                optimizer = optim.Adam(model.parameters(), lr=0.002)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
                train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=20)
                model.eval()
                with torch.no_grad():
                    val_tensor = torch.FloatTensor(X_val).to(device)
                    test_tensor = torch.FloatTensor(X_test).to(device)
                    val_pred = model(val_tensor).squeeze().cpu().numpy()
                    test_pred = model(test_tensor).squeeze().cpu().numpy()
                    base_preds_val.append(val_pred)
                    base_preds_test.append(test_pred)
                successful_models.append(model)
            except RuntimeError as e:
                if "mat1 and mat2 shapes cannot be multiplied" in str(e):
                    print(f"Warning: Skipping {model_name} due to shape mismatch in fold {fold + 1}: {str(e)}")
                else:
                    print(f"Error training {model_name} in fold {fold + 1}: {str(e)}")
                continue
        
        if not successful_models:
            print(f"Error: No successful base models in fold {fold + 1}. Skipping HiTCLe evaluation.")
            continue
        
        base_preds_val = np.column_stack(base_preds_val)
        base_preds_test = np.column_stack(base_preds_test)
        
        # Verify predictions
        if base_preds_val.shape[1] == 0 or base_preds_test.shape[1] == 0:
            print(f"Error: Empty predictions in fold {fold + 1}. Skipping HiTCLe evaluation.")
            continue
        
        # Train meta-model for HiTCLe
        meta_input_dim = len(successful_models)  # Number of successful base models
        print(f"Meta-model input dimension: {meta_input_dim} for fold {fold + 1}")
        meta_model = HighwayModel(input_dim=meta_input_dim).to(device)
        optimizer = optim.Adam(meta_model.parameters(), lr=0.002)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
        meta_dataset = TensorDataset(torch.FloatTensor(base_preds_val), torch.FloatTensor(y_val))
        meta_loader = DataLoader(meta_dataset, batch_size=128, shuffle=True)
        print("Training HiTCLe Meta-Model")
        train_model(meta_model, meta_loader, criterion, optimizer, scheduler, num_epochs=20)
        
        # Evaluate HiTCLe
        meta_model.eval()
        with torch.no_grad():
            test_tensor = torch.FloatTensor(base_preds_test).to(device)
            outputs = meta_model(test_tensor).squeeze()
            preds = (outputs > 0.5).float().cpu().numpy()
        
        acc, f1, prec, rec = accuracy_score(y_test, preds), f1_score(y_test, preds, zero_division=0), \
                             precision_score(y_test, preds, zero_division=0), recall_score(y_test, preds, zero_division=0)
        metrics_hitcle['accuracy'].append(acc)
        metrics_hitcle['f1'].append(f1)
        metrics_hitcle['precision'].append(prec)
        metrics_hitcle['recall'].append(rec)
        print(f"HiTCLe Fold {fold + 1} - Accuracy: {acc:.4f}, F1: {f1:.4f}")
    
    except Exception as e:
        print(f"Error in fold {fold + 1}: {str(e)}. Skipping fold.")
        continue


Fold 1/10
Training HiLeModel
Epoch 5/20, Avg Loss: 0.7012
Epoch 10/20, Avg Loss: 0.6852
Epoch 15/20, Avg Loss: 0.6647
Epoch 20/20, Avg Loss: 0.6422
HiLeModel Fold 1 - Accuracy: 0.5192, F1: 0.6835
Training HighwayModel for fold 1
Epoch 5/20, Avg Loss: 0.4854
Epoch 10/20, Avg Loss: 0.4109
Epoch 15/20, Avg Loss: 0.3817
Epoch 20/20, Avg Loss: 0.3510
Training LeNetModel for fold 1
Epoch 5/20, Avg Loss: 0.4773
Epoch 10/20, Avg Loss: 0.4482
Epoch 15/20, Avg Loss: 0.4457
Epoch 20/20, Avg Loss: 0.4411
Training TCNModel for fold 1
Meta-model input dimension: 2 for fold 1
Training HiTCLe Meta-Model
Epoch 5/20, Avg Loss: 0.6647
Epoch 10/20, Avg Loss: 0.6325
Epoch 15/20, Avg Loss: 0.5874
Epoch 20/20, Avg Loss: 0.5377
HiTCLe Fold 1 - Accuracy: 0.7596, F1: 0.8062

Fold 2/10
Training HiLeModel
Epoch 5/20, Avg Loss: 0.6291
Epoch 10/20, Avg Loss: 0.6093
Epoch 15/20, Avg Loss: 0.5836
Epoch 20/20, Avg Loss: 0.5686
HiLeModel Fold 2 - Accuracy: 0.6827, F1: 0.7273
Training HighwayModel for fold 2
Epoch 5/20

In [165]:

# Check if any folds were successful
if not metrics_hile['accuracy']:
    raise ValueError("No successful folds completed. Check data or StratifiedKFold configuration.")

In [170]:
# # Compute average metrics
# print("\nHi-Le Model Average Metrics (10-Fold CV):")
# for metric, values in metrics_hile.items():
#     print(f"{metric.capitalize()}: {np.mean(values):.4f} ± {np.std(values):.4f}")

# print("\nHiTCLe Model Average Metrics (10-Fold CV):")
# for metric, values in metrics_hitcle.items():
#     print(f"{metric.capitalize()}: {np.mean(values):.4f} ± {np.std(values):.4f}")

# # LIME Analysis for Hi-Le Model
# hile_model = HiLeModel(input_dim=X.shape[1]).to(device)
# train_dataset = TensorDataset(torch.FloatTensor(X_balanced), torch.FloatTensor(y_balanced))
# train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
# train_model(hile_model, train_loader, nn.BCELoss(), optim.Adam(hile_model.parameters(), lr=0.002), 
#             optim.lr_scheduler.ReduceLROnPlateau(optim.Adam(hile_model.parameters(), lr=0.002), mode='min', factor=0.5, patience=3), num_epochs=20)

# def hile_predict_proba(X):
#     hile_model.eval()
#     X_tensor = torch.FloatTensor(X).to(device)
#     with torch.no_grad():
#         outputs = hile_model(X_tensor).squeeze().cpu().numpy()
#     if outputs.ndim == 0:
#         outputs = np.array([outputs])
#     return np.column_stack([1 - outputs, outputs])

# explainer = LimeTabularExplainer(
#     training_data=X_balanced,
#     feature_names=columns[:-1],
#     class_names=['Non-Diabetic', 'Diabetic'],
#     mode='classification'
# )

# np.random.seed(42)
# sample_idx = np.random.randint(0, len(X_balanced))
# exp = explainer.explain_instance(
#     data_row=X_balanced[sample_idx],
#     predict_fn=hile_predict_proba,
#     num_features=8
# )

# plt.figure(figsize=(10, 6))
# exp.as_pyplot_figure()
# plt.title(f"LIME Explanation for Hi-Le Model (Instance {sample_idx})")
# plt.tight_layout()
# plt.savefig("lime_hile.png")

# # LIME Analysis for HiTCLe Model
# base_models = [
#     (HighwayModel(input_dim=X.shape[1]).to(device), "HighwayModel"),
#     (LeNetModel(input_dim=X.shape[1]).to(device), "LeNetModel"),
#     (TCNModel(input_dim=X.shape[1], filters=32).to(device), "TCNModel")
# ]
# successful_base_models = []

# for model, model_name in base_models:
#     try:
#         train_model(model, train_loader, nn.BCELoss(), optim.Adam(model.parameters(), lr=0.002), 
#                     optim.lr_scheduler.ReduceLROnPlateau(optim.Adam(model.parameters(), lr=0.002), mode='min', factor=0.5, patience=3), num_epochs=20)
#         successful_base_models.append(model)
#     except RuntimeError as e:
#         if "mat1 and mat2 shapes cannot be multiplied" in str(e):
#             print(f"Warning: Excluding {model_name} from LIME analysis due to shape mismatch. Continuing with other models.")
#         else:
#             raise e

# meta_input_dim = len(successful_base_models)
# meta_model = HighwayModel(input_dim=meta_input_dim).to(device)
# base_preds = np.column_stack([
#     model(torch.FloatTensor(X_balanced).to(device)).squeeze().cpu().detach().numpy()
#     for model in successful_base_models
# ])
# meta_dataset = TensorDataset(torch.FloatTensor(base_preds), torch.FloatTensor(y_balanced))
# meta_loader = DataLoader(meta_dataset, batch_size=128, shuffle=True)
# train_model(meta_model, meta_loader, nn.BCELoss(), optim.Adam(meta_model.parameters(), lr=0.002), 
#             optim.lr_scheduler.ReduceLROnPlateau(optim.Adam(meta_model.parameters(), lr=0.002), mode='min', factor=0.5, patience=3), num_epochs=20)

# def hitcle_predict_proba(X):
#     base_preds = np.column_stack([
#         model(torch.FloatTensor(X).to(device)).squeeze().cpu().detach().numpy()
#         for model in successful_base_models
#     ])
#     meta_model.eval()
#     with torch.no_grad():
#         outputs = meta_model(torch.FloatTensor(base_preds).to(device)).squeeze().cpu().numpy()
#     if outputs.ndim == 0:
#         outputs = np.array([outputs])
#     return np.column_stack([1 - outputs, outputs])

# exp_hitcle = explainer.explain_instance(
#     data_row=X_balanced[sample_idx],
#     predict_fn=hitcle_predict_proba,
#     num_features=8
# )

# plt.figure(figsize=(10, 6))
# exp_hitcle.as_pyplot_figure()
# plt.title(f"LIME Explanation for HiTCLe Model (Instance {sample_idx})")
# plt.tight_layout()
# plt.savefig("lime_hitcle.png")

# # Plot model performance comparison
# epochs = [5, 10, 20]
# hile_acc = [np.mean(metrics_hile['accuracy'])] * len(epochs)
# hitcle_acc = [np.mean(metrics_hitcle['accuracy'])] * len(epochs)
# plt.figure()
# plt.plot(epochs, hile_acc, label='Hi-Le', marker='o')
# plt.plot(epochs, hitcle_acc, label='HiTCLe', marker='s')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Model Performance Comparison')
# plt.legend()
# plt.grid(True)
# plt.savefig("performance_comparison.png")
# Compute average metrics
# Compute average metrics
# Compute average metrics
print("\nHi-Le Model Average Metrics (10-Fold CV):")
for metric, values in metrics_hile.items():
    print(f"{metric.capitalize()}: {np.mean(values):.4f} ± {np.std(values):.4f}")

print("\nHiTCLe Model Average Metrics (10-Fold CV):")
for metric, values in metrics_hitcle.items():
    print(f"{metric.capitalize()}: {np.mean(values):.4f} ± {np.std(values):.4f}")

# LIME Analysis for Hi-Le Model
hile_model = HiLeModel(input_dim=X.shape[1]).to(device)
train_dataset = TensorDataset(torch.FloatTensor(X_balanced), torch.FloatTensor(y_balanced))
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
train_model(hile_model, train_loader, nn.BCELoss(), optim.Adam(hile_model.parameters(), lr=0.002), 
            optim.lr_scheduler.ReduceLROnPlateau(optim.Adam(hile_model.parameters(), lr=0.002), mode='min', factor=0.5, patience=3), num_epochs=20)

def hile_predict_proba(X):
    hile_model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        outputs = hile_model(X_tensor).squeeze().cpu().numpy()
    if outputs.ndim == 0:
        outputs = np.array([outputs])
    return np.column_stack([1 - outputs, outputs])

explainer = LimeTabularExplainer(
    training_data=X_balanced,
    feature_names=columns[:-1],
    class_names=['Non-Diabetic', 'Diabetic'],
    mode='classification'
)

np.random.seed(42)
sample_idx = np.random.randint(0, len(X_balanced))
exp = explainer.explain_instance(
    data_row=X_balanced[sample_idx],
    predict_fn=hile_predict_proba,
    num_features=8
)

plt.figure(figsize=(10, 6))
exp.as_pyplot_figure()
plt.title(f"LIME Explanation for Hi-Le Model (Instance {sample_idx})")
plt.tight_layout()
plt.savefig("lime_hile.png")

# LIME Analysis for HiTCLe Model
base_models = [
    (HighwayModel(input_dim=X.shape[1]).to(device), "HighwayModel"),
    (LeNetModel(input_dim=X.shape[1]).to(device), "LeNetModel"),
    (TCNModel(input_dim=X.shape[1], filters=32).to(device), "TCNModel")
]
successful_base_models = []

for model, model_name in base_models:
    try:
        train_model(model, train_loader, nn.BCELoss(), optim.Adam(model.parameters(), lr=0.002), 
                    optim.lr_scheduler.ReduceLROnPlateau(optim.Adam(model.parameters(), lr=0.002), mode='min', factor=0.5, patience=3), num_epochs=20)
        successful_base_models.append(model)
    except RuntimeError as e:
        if "mat1 and mat2 shapes cannot be multiplied" in str(e):
            print(f"Warning: Excluding {model_name} from LIME analysis due to shape mismatch. Continuing with other models.")
        else:
            raise e

meta_input_dim = len(successful_base_models)
meta_model = HighwayModel(input_dim=meta_input_dim).to(device)
base_preds = np.column_stack([
    model(torch.FloatTensor(X_balanced).to(device)).squeeze().cpu().detach().numpy()
    for model in successful_base_models
])
meta_dataset = TensorDataset(torch.FloatTensor(base_preds), torch.FloatTensor(y_balanced))
meta_loader = DataLoader(meta_dataset, batch_size=128, shuffle=True)
train_model(meta_model, meta_loader, nn.BCELoss(), optim.Adam(meta_model.parameters(), lr=0.002), 
            optim.lr_scheduler.ReduceLROnPlateau(optim.Adam(meta_model.parameters(), lr=0.002), mode='min', factor=0.5, patience=3), num_epochs=20)

def hitcle_predict_proba(X):
    base_preds = np.column_stack([
        model(torch.FloatTensor(X).to(device)).squeeze().cpu().detach().numpy()
        for model in successful_base_models
    ])
    meta_model.eval()
    with torch.no_grad():
        outputs = meta_model(torch.FloatTensor(base_preds).to(device)).squeeze().cpu().numpy()
    if outputs.ndim == 0:
        outputs = np.array([outputs])
    return np.column_stack([1 - outputs, outputs])

exp_hitcle = explainer.explain_instance(
    data_row=X_balanced[sample_idx],
    predict_fn=hitcle_predict_proba,
    num_features=8
)

plt.figure(figsize=(10, 6))
exp_hitcle.as_pyplot_figure()
plt.title(f"LIME Explanation for HiTCLe Model (Instance {sample_idx})")
plt.tight_layout()
plt.savefig("lime_hitcle.png")

# Plot model performance comparison
epochs = [5, 10, 20]
hile_acc = [np.mean(metrics_hile['accuracy'])] * len(epochs)
hitcle_acc = [np.mean(metrics_hitcle['accuracy'])] * len(epochs)
plt.figure()
plt.plot(epochs, hile_acc, label='Hi-Le', marker='o')
plt.plot(epochs, hitcle_acc, label='HiTCLe', marker='s')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.legend()
plt.grid(True)
plt.savefig("performance_comparison.png")
import pickle
model_save_path = "model.pkl"
model_data = {
    "input_dim": X.shape[1],
    "meta_input_dim": meta_input_dim,
    "hile_state_dict": hile_model.state_dict(),
    "hitcle_base_state_dicts": [(model.state_dict(), model.__class__.__name__) for model in successful_base_models],
    "hitcle_meta_state_dict": meta_model.state_dict()
}
with open(model_save_path, "wb") as f:
    pickle.dump(model_data, f)
print(f"Models saved to {model_save_path}")


Hi-Le Model Average Metrics (10-Fold CV):
Accuracy: 0.6961 ± 0.1336
F1: 0.6904 ± 0.2370
Precision: 0.6510 ± 0.2507
Recall: 0.7657 ± 0.2696

HiTCLe Model Average Metrics (10-Fold CV):
Accuracy: 0.7810 ± 0.0368
F1: 0.8148 ± 0.0305
Precision: 0.7263 ± 0.0372
Recall: 0.9309 ± 0.0514
Epoch 5/20, Avg Loss: 0.6296
Epoch 10/20, Avg Loss: 0.6009
Epoch 15/20, Avg Loss: 0.5849
Epoch 20/20, Avg Loss: 0.5561
Epoch 5/20, Avg Loss: 0.4782
Epoch 10/20, Avg Loss: 0.3826
Epoch 15/20, Avg Loss: 0.4150
Epoch 20/20, Avg Loss: 0.3821
Epoch 5/20, Avg Loss: 0.4987
Epoch 10/20, Avg Loss: 0.4240
Epoch 15/20, Avg Loss: 0.4372
Epoch 20/20, Avg Loss: 0.4597
Epoch 5/20, Avg Loss: 0.4437
Epoch 10/20, Avg Loss: 0.3682
Epoch 15/20, Avg Loss: 0.3542
Epoch 20/20, Avg Loss: 0.3620
Models saved to model.pkl


import pickle
model_save_path = "model.pkl"
model_data = {
    "input_dim": X.shape[1],
    "meta_input_dim": meta_input_dim,
    "hile_state_dict": hile_model.state_dict(),
    "hitcle_base_state_dicts": [(model.state_dict(), model.__class__.__name__) for model in successful_base_models],
    "hitcle_meta_state_dict": meta_model.state_dict()
}
with open(model_save_path, "wb") as f:
    pickle.dump(model_data, f)
print(f"Models saved to {model_save_path}")